### Import Dependencies

In [15]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Float

In [16]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect, func

In [17]:
engine = create_engine()
inspector = inspect(engine)

TypeError: create() missing 1 required positional argument: 'name_or_url'

In [ ]:
Base = automap_base()
Base.prepare(engine, reflect=True)
Base.classes.keys()
session = Session(engine)

In [14]:
Base = declarative_base()

### Extract CSVs into DataFrames

In [2]:
athlete_info = pd.read_csv("olympic history/athlete_events.csv")
country_info = pd.read_csv("olympic history/noc_regions.csv")
olympic_df = pd.merge(athlete_info, country_info, how="left", on = "NOC")
olympic_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,NaN


In [3]:
olympic_df["medal_won"] = (olympic_df["Medal"] != "NaN")

poverty_df = pd.read_csv("global poverty/MPI_national.csv")
poverty_df.head()

,ISO,Country,MPI Urban,Headcount Ratio Urban,Intensity of Deprivation Urban,MPI Rural,Headcount Ratio Rural,Intensity of Deprivation Rural
0,KAZ,Kazakhstan,0.000,0.0,33.3,0.000,0.09,33.3
1,SRB,Serbia,0.000,0.1,41.4,0.002,0.50,40.3
2,KGZ,Kyrgyzstan,0.000,0.1,40.2,0.003,0.70,37.1
3,TUN,Tunisia,0.000,0.1,35.6,0.012,3.18,38.7
4,ARM,Armenia,0.001,0.2,33.3,0.001,0.39,36.9


In [19]:
olympic_df["medal_won"] = (olympic_df["Medal"] == "Gold")
olympic_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes,medal_won
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN,False
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN,False
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN,False
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN,True
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,NaN,False


In [5]:
# Rename column "region"
olympic_df = olympic_df.rename(columns = {"region":"Country"})
olympic_df.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,Country,notes,medal_won
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN,True
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN,True
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN,True
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN,True
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,NaN,True


In [27]:
# Aggregate data by region 
by_country = olympic_df.groupby("Country")
medal_count = by_country["Medal"].count()
agg_olympic_df = pd.DataFrame({"Number of Medals Won": medal_count})


# Merge dataframes 
combined_df = pd.merge(agg_olympic_df, poverty_df, how = "inner", on = "Country")

# Extract only relevant columns from poverty dataframe 

In [29]:
combined_df = pd.merge(agg_olympic_df, poverty_df, how = "inner", on = "Country")
combined_df.head()

,Country,Number of Medals Won,ISO,MPI Urban,Headcount Ratio Urban,Intensity of Deprivation Urban,MPI Rural,Headcount Ratio Rural,Intensity of Deprivation Rural
0,Afghanistan,2,AFG,0.132,28.8,45.8,0.347,64.66,53.6
1,Albania,0,ALB,0.003,0.8,37.7,0.007,1.87,37.7
2,Algeria,17,DZA,0.003,0.7,36.9,0.011,2.70,40.0
3,Armenia,16,ARM,0.001,0.2,33.3,0.001,0.39,36.9
4,Azerbaijan,44,AZE,0.009,2.5,37.1,0.035,8.80,40.2


In [32]:
#Extract only relevant columns
combined_df2 = combined_df[["Country", "Number of Medals Won", "MPI Urban", "MPI Rural", "Headcount Ratio Urban", "Headcount Ratio Rural"]]
combined_df2.head()

,Country,Number of Medals Won,MPI Urban,MPI Rural,Headcount Ratio Urban,Headcount Ratio Rural
0,Afghanistan,2,0.132,0.347,28.8,64.66
1,Albania,0,0.003,0.007,0.8,1.87
2,Algeria,17,0.003,0.011,0.7,2.70
3,Armenia,16,0.001,0.001,0.2,0.39
4,Azerbaijan,44,0.009,0.035,2.5,8.80


### Transform premise DataFrame

### Transform county DataFrame

### Create database connection

### Load DataFrames into database